In [11]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium import webdriver 
import time 
from bs4 import BeautifulSoup

# pages_to_scrape = 3

url = 'https://www.tripadvisor.com/Hotel_Review-g469404-d301649-Reviews-The_Legian_Seminyak_Bali-Seminyak_Kuta_District_Bali.html'
options = Options()
driver = webdriver.Chrome(options=options) 
driver.get(url)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="WAllg _T"]')))
time.sleep(10)

df_data = pd.DataFrame(columns = ['title', 'rating', 'date_of_stay', 'review'])

# for i in range(0,pages_to_scrape):
while True:
    time.sleep(2)
    #read more
    while True:
        try:
            expand_element = driver.find_element(By.XPATH, '//div[@class="TnInx" and @data-test-target="expand-review"]/span[@class="Ignyf _S Z"]')
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="TnInx" and @data-test-target="expand-review"]/span[@class="Ignyf _S Z"]')))
            expand_element.click()
            break
        except (ElementClickInterceptedException, StaleElementReferenceException):
            time.sleep(2)
    
    time.sleep(10)
    container = driver.find_elements(By.XPATH, '//div[@class="WAllg _T"]')
    
    for j in range(len(container)): 
        
        try:
            title_element = container[j].find_element(By.XPATH, ".//div[contains(@data-test-target, 'review-title')]")
            driver.execute_script("arguments[0].scrollIntoView();", title_element)
            title = title_element.text
        except (StaleElementReferenceException, NoSuchElementException):
            title = ""

        try:
            review_element = container[j].find_element(By.XPATH, ".//div[contains(@class, 'fIrGe')]//span[contains(@class, 'QewHA')]")
            driver.execute_script("arguments[0].scrollIntoView();", review_element)
            review = review_element.text
        except (StaleElementReferenceException, NoSuchElementException):
            review = ""
            
        try:
            soup = BeautifulSoup(container[j].get_attribute('innerHTML'), 'html.parser')
            date_element = soup.find('span', {'class': 'teHYY _R Me S4 H3'}).find('span', {'class': 'usajM'}).next_sibling.strip()
            date_of_stay = date_element.replace("Date of stay: ","")
        except (AttributeError, ValueError):
            date_of_stay = ""
            
        try:
            soup = BeautifulSoup(container[j].get_attribute('innerHTML'), 'html.parser')
            rating = soup.select_one('span.ui_bubble_rating')['class'][1][7:]
        except (AttributeError, ValueError):
            date_of_stay = ""

        df_data.loc[len(df_data)] = [title, rating, date_of_stay, review]

    try:
        driver.find_element(by=By.CSS_SELECTOR, value='.ui_button.nav.next.primary').click()
    except ElementClickInterceptedException:
        break
        
driver.quit()

In [12]:
df_data

,title,rating,date_of_stay,review
0,Perfection,50,April 2023,"From the very first moment you arrive, you can..."
1,5 star rating is just UNDERstated !,50,April 2023,May not be the newest property but still very ...
2,The best hotel in Bali.,50,April 2023,"Still the best hotel in Bali.\nService, locati..."
3,Perfect in every way…,50,March 2023,We stayed here for five nights and absolutely ...
4,Best place in Bali,50,April 2023,This place was fantastic. Everything from the ...
...,...,...,...,...
1854,Fantastic in every sense!,50,September 2005,"This is, by far, the Best Hotel I've ever stay..."
1855,Paradise - but not for children,50,August 2005,We stayed at The Club Legian for 10 nights and...
1856,The Legian Was Superb,50,July 2005,The Legian was the perfect retreat. Aside from...
1857,Unforgetable 30th B'day trip,50,March 2004,"Where should I begin, my husband and I stayed ..."


In [13]:
df_data.to_csv("TheLegianSeminyakBali.csv")